<a href="https://colab.research.google.com/github/LuHellerKP/SMA_PUCRS/blob/main/SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import matplotlib.pyplot as plt

class Event:
    def __init__(self, time, event_type):
        self.time = time      # Tempo do evento
        self.event_type = event_type  # 'arrival' ou 'departure'

    def __lt__(self, other):
        return self.time < other.time

class QueueSimulator:
    def __init__(self, num_servers, queue_capacity, arrival_range, service_range, random_numbers):
        self.num_servers = num_servers
        self.queue_capacity = queue_capacity
        self.arrival_range = arrival_range  # (a, b) para chegadas
        self.service_range = service_range  # (a, b) para serviços
        self.random_numbers = random_numbers  # Lista de números pseudoaleatórios
        self.current_time = 0.0
        self.event_queue = []
        self.queue = []
        self.num_in_service = 0
        self.state_time = {}  # Tempo acumulado em cada estado
        self.last_event_time = 0.0
        self.lost_customers = 0
        self.total_customers = 0
        self.random_index = 0

        # Inicializar estados
        self.state_time = {i: 0.0 for i in range(queue_capacity + num_servers + 1)}
        # Agendar a primeira chegada
        first_arrival = 2.0
        heapq.heappush(self.event_queue, Event(first_arrival, 'arrival'))

    def get_next_random(self):
        if self.random_index >= len(self.random_numbers):
            raise Exception("Esgotaram os números pseudoaleatórios disponíveis.")
        x = self.random_numbers[self.random_index]
        self.random_index += 1
        return x

    def generate_time(self, a, b):
        x = self.get_next_random()
        return a + (b - a) * x

    def run(self, max_randoms):
        while self.random_index < max_randoms and self.event_queue:
            event = heapq.heappop(self.event_queue)
            # Atualizar o tempo acumulado para o estado anterior
            time_diff = event.time - self.last_event_time
            current_state = self.num_in_service + len(self.queue)
            self.state_time[current_state] += time_diff
            self.current_time = event.time
            self.last_event_time = event.time

            if event.event_type == 'arrival':
                self.handle_arrival()
            elif event.event_type == 'departure':
                self.handle_departure()

        self.total_time = self.current_time

    def handle_arrival(self):
        self.total_customers += 1
        # Gerar próximo tempo de chegada
        next_arrival_time = self.current_time + self.generate_time(*self.arrival_range)
        if self.random_index < len(self.random_numbers):
            heapq.heappush(self.event_queue, Event(next_arrival_time, 'arrival'))

        # Verificar se pode adicionar à fila
        if self.num_in_service < self.num_servers:
            # Atender imediatamente
            service_time = self.generate_time(*self.service_range)
            departure_time = self.current_time + service_time
            heapq.heappush(self.event_queue, Event(departure_time, 'departure'))
            self.num_in_service += 1
        else:
            if len(self.queue) < self.queue_capacity:
                self.queue.append(self.current_time)
            else:
                self.lost_customers += 1  # Cliente perdido

    def handle_departure(self):
        self.num_in_service -= 1
        if self.queue:
            # Atender o próximo cliente na fila
            arrival_time = self.queue.pop(0)
            wait_time = self.current_time - arrival_time
            service_time = self.generate_time(*self.service_range)
            departure_time = self.current_time + service_time
            heapq.heappush(self.event_queue, Event(departure_time, 'departure'))
            self.num_in_service += 1

    def get_statistics(self):
        # Distribuição de Probabilidades
        total_time = sum(self.state_time.values())
        probability_distribution = {state: time / total_time for state, time in self.state_time.items()}

        # População Média
        average_population = sum(state * time for state, time in self.state_time.items()) / total_time

        return {
            'probability_distribution': probability_distribution,
            'average_population': average_population,
            'lost_customers': self.lost_customers,
            'total_time': self.total_time
        }


In [2]:
import random

def linear_congruential_generator(n, X0, a, c, M):
    numbers = []
    X = X0
    for _ in range(n):
        X = (a * X + c) % M
        numbers.append(X / M)  # Normalização para o intervalo [0, 1]
    return numbers

# Definição dos parâmetros (use os mesmos que você utilizou anteriormente)
X0 = 7
a = 1664525
c = 1013904223
M = 2**32

# Gerar 100.000 números
random_numbers = linear_congruential_generator(100000, X0, a, c, M)


In [3]:
# Parâmetros das filas
arrival_range_GG1_5 = (2.0, 5.0)  # Chegadas entre 2 e 5 minutos
service_range_GG1_5 = (3.0, 5.0)  # Atendimento entre 3 e 5 minutos

arrival_range_GG2_5 = (2.0, 5.0)
service_range_GG2_5 = (3.0, 5.0)

# Simulação G/G/1/5
sim_GG1_5 = QueueSimulator(
    num_servers=1,
    queue_capacity=5,
    arrival_range=arrival_range_GG1_5,
    service_range=service_range_GG1_5,
    random_numbers=random_numbers.copy()  # Copia para evitar conflito
)
sim_GG1_5.run(max_randoms=100000)
stats_GG1_5 = sim_GG1_5.get_statistics()

# Simulação G/G/2/5
sim_GG2_5 = QueueSimulator(
    num_servers=2,
    queue_capacity=5,
    arrival_range=arrival_range_GG2_5,
    service_range=service_range_GG2_5,
    random_numbers=random_numbers.copy()
)
sim_GG2_5.run(max_randoms=100000)
stats_GG2_5 = sim_GG2_5.get_statistics()

# Exibindo as Estatísticas
def display_statistics(stats, queue_type):
    print(f"--- Estatísticas para {queue_type} ---")
    print("Distribuição de Probabilidades:")
    for state, prob in sorted(stats['probability_distribution'].items()):
        print(f"Estado {state}: {prob:.4f}")
    print(f"População Média: {stats['average_population']:.4f} clientes")
    print(f"Clientes Perdidos: {stats['lost_customers']}")
    print(f"Tempo Total da Simulação: {stats['total_time']:.2f} minutos\n")

display_statistics(stats_GG1_5, "G/G/1/5")
display_statistics(stats_GG2_5, "G/G/2/5")


--- Estatísticas para G/G/1/5 ---
Distribuição de Probabilidades:
Estado 0: 0.0000
Estado 1: 0.0000
Estado 2: 0.0003
Estado 3: 0.0017
Estado 4: 0.0399
Estado 5: 0.5234
Estado 6: 0.4347
População Média: 5.3903 clientes
Clientes Perdidos: 6746
Tempo Total da Simulação: 186536.16 minutos

--- Estatísticas para G/G/2/5 ---
Distribuição de Probabilidades:
Estado 0: 0.0636
Estado 1: 0.7315
Estado 2: 0.2042
Estado 3: 0.0007
Estado 4: 0.0000
Estado 5: 0.0000
Estado 6: 0.0000
Estado 7: 0.0000
População Média: 1.1419 clientes
Clientes Perdidos: 0
Tempo Total da Simulação: 175072.58 minutos

